# Full Script

This block drops/creates the timetable schema, switches the search path, and defines all base tables for rooms, classes, instructors, students, and their linking/option tables (class_instructors, class_room_options, class_time_options). It also models constraint metadata (constraints, constraint_classes) and room-sharing structures. Primary keys and foreign keys establish referential integrity, and composite PKs on many-to-many/link tables prevent duplicates while encoding the scheduling domain cleanly.

In [ ]:
DROP SCHEMA IF EXISTS timetable CASCADE;
CREATE SCHEMA timetable;
SET search_path = timetable;

CREATE TABLE rooms (
  room_id text PRIMARY KEY,
  capacity integer,
  location_x integer,
  location_y integer,
  has_constraints boolean
);

CREATE TABLE room_sharing_patterns (
  room_id text PRIMARY KEY REFERENCES rooms(room_id),
  unit_slots integer,
  free_for_all_char text,
  not_available_char text,
  pattern_text text
);

CREATE TABLE room_sharing_departments (
  room_id text REFERENCES rooms(room_id),
  digit_char text,
  department_id text,
  PRIMARY KEY (room_id, digit_char, department_id)
);

CREATE TABLE classes (
  class_id text PRIMARY KEY,
  offering_id text,
  config_id  text,
  subpart_id text,
  committed boolean,
  class_limit integer,
  scheduler integer,
  dates_mask text
);

CREATE TABLE class_instructors (
  class_id text REFERENCES classes(class_id),
  instructor_id text,
  PRIMARY KEY (class_id, instructor_id)
);

CREATE TABLE class_room_options (
  class_id text REFERENCES classes(class_id),
  room_id  text REFERENCES rooms(room_id),
  pref double precision,
  PRIMARY KEY (class_id, room_id)
);

CREATE TABLE class_time_options (
  class_id text REFERENCES classes(class_id),
  days_mask text,
  start_slot integer,
  length_slots integer,
  pref  double precision,
  PRIMARY KEY (class_id, days_mask, start_slot, length_slots)
);

CREATE TABLE constraints (
  pk integer PRIMARY KEY,
  external_id text,
  type text,
  pref_raw text,
  pref_numeric double precision
);

CREATE TABLE constraint_classes (
  constraint_pk integer REFERENCES constraints(pk),
  order_index integer,
  class_id text REFERENCES classes(class_id),
  PRIMARY KEY (constraint_pk, order_index)
);

CREATE TABLE students (
  student_id text PRIMARY KEY
);

CREATE TABLE IF NOT EXISTS instructors (
  instructor_id VARCHAR(50) PRIMARY KEY
);

CREATE TABLE student_offerings (
  student_id  text REFERENCES students(student_id),
  offering_id text,
  weight  double precision,
  PRIMARY KEY (student_id, offering_id)
);

CREATE TABLE student_classes (
  student_id text REFERENCES students(student_id),
  class_id text REFERENCES classes(class_id),
  PRIMARY KEY (student_id, class_id)
);

CREATE TABLE student_prohibited_classes (
  student_id text REFERENCES students(student_id),
  class_id text REFERENCES classes(class_id),
  PRIMARY KEY (student_id, class_id)
);

SyntaxError: invalid syntax (ipython-input-1165982166.py, line 1)

This block defines timetable.load_csv_dedup(target_table, csv_path, col_list, pk_cols), a reusable postgreSQL procedure. It creates a staging table that mirrors the target, bulk-loads a CSV via COPY, and inserts into the target using SELECT DISTINCT ON (pk_cols) plus ON CONFLICT DO NOTHING to deduplicate by the provided primary key column set. It then drops the staging table, giving us a safe CSV loader per table.

In [ ]:

CREATE OR REPLACE PROCEDURE timetable.load_csv_dedup(
  target_table regclass,
  csv_path text,
  col_list text,
  pk_cols text
)
LANGUAGE plpgsql
AS $$
DECLARE
  stg_name text := quote_ident(split_part(target_table::text, '.', 2) || '_stg');
  tgt_name text := target_table::text;
  sql text;
BEGIN
  EXECUTE format('DROP TABLE IF EXISTS %s', stg_name);
  EXECUTE format('CREATE TEMP TABLE %s (LIKE %s INCLUDING DEFAULTS)', stg_name, tgt_name);

  sql := format($f$COPY %s (%s) FROM %L CSV HEADER$f$, stg_name, col_list, csv_path);
  EXECUTE sql;

  sql := format($f$
    INSERT INTO %s (%s)
    SELECT DISTINCT ON (%s) %s
    FROM %s
    ORDER BY %s
    ON CONFLICT DO NOTHING
  $f$, tgt_name, col_list, pk_cols, col_list, stg_name, pk_cols);
  EXECUTE sql;

  EXECUTE format('DROP TABLE %s', stg_name);
END;
$$;


Here, we set a timetable.base_path with the dataset folder and call load_csv_dedup for every table, passing the table name, CSV path, column list, and PK columns. This systematically fills the schema (rooms, sharing patterns/departments, classes, instructors, room/time options, constraints, students, and enrollment/demand) while guaranteeing uniqueness and FK-compatible ordering (parents first, then children).

In [ ]:
SET timetable.base_path = '/absolute_path';

CALL timetable.load_csv_dedup(
  'timetable.rooms',
  current_setting('timetable.base_path') || '/rooms.csv',
  'room_id,capacity,location_x,location_y,has_constraints',
  'room_id'
);

CALL timetable.load_csv_dedup(
  'timetable.room_sharing_patterns',
  current_setting('timetable.base_path') || '/room_sharing_patterns.csv',
  'room_id,unit_slots,free_for_all_char,not_available_char,pattern_text',
  'room_id'
);

CALL timetable.load_csv_dedup(
  'timetable.room_sharing_departments',
  current_setting('timetable.base_path') || '/room_sharing_departments.csv',
  'room_id,digit_char,department_id',
  'room_id,digit_char,department_id'
);

CALL timetable.load_csv_dedup(
  'timetable.classes',
  current_setting('timetable.base_path') || '/classes.csv',
  'class_id,offering_id,config_id,subpart_id,committed,class_limit,scheduler,dates_mask',
  'class_id'
);

CALL timetable.load_csv_dedup(
  'timetable.class_instructors',
  current_setting('timetable.base_path') || '/class_instructors.csv',
  'class_id,instructor_id',
  'class_id,instructor_id'
);

CALL timetable.load_csv_dedup(
  'timetable.class_room_options',
  current_setting('timetable.base_path') || '/class_room_options.csv',
  'class_id,room_id,pref',
  'class_id,room_id'
);

CALL timetable.load_csv_dedup(
  'timetable.class_time_options',
  current_setting('timetable.base_path') || '/class_time_options.csv',
  'class_id,days_mask,start_slot,length_slots,pref',
  'class_id,days_mask,start_slot,length_slots'
);

CALL timetable.load_csv_dedup(
  'timetable.instructors',
  current_setting('timetable.base_path') || '/instructors.csv',
  'instructor_id',
  'instructor_id'
);

CALL timetable.load_csv_dedup(
  'timetable.constraints',
  current_setting('timetable.base_path') || '/constraints.csv',
  'pk,external_id,type,pref_raw,pref_numeric',
  'pk'
);

CALL timetable.load_csv_dedup(
  'timetable.constraint_classes',
  current_setting('timetable.base_path') || '/constraint_classes.csv',
  'constraint_pk,order_index,class_id',
  'constraint_pk,order_index'
);

CALL timetable.load_csv_dedup(
  'timetable.students',
  current_setting('timetable.base_path') || '/students.csv',
  'student_id',
  'student_id'
);

CALL timetable.load_csv_dedup(
  'timetable.student_offerings',
  current_setting('timetable.base_path') || '/student_offerings.csv',
  'student_id,offering_id,weight',
  'student_id,offering_id'
);

CALL timetable.load_csv_dedup(
  'timetable.student_classes',
  current_setting('timetable.base_path') || '/student_classes.csv',
  'student_id,class_id',
  'student_id,class_id'
);

CALL timetable.load_csv_dedup(
  'timetable.student_prohibited_classes',
  current_setting('timetable.base_path') || '/student_prohibited_classes.csv',
  'student_id,class_id',
  'student_id,class_id'
);

SELECT 'rooms' AS t, COUNT(*) FROM timetable.rooms
UNION ALL SELECT 'classes', COUNT(*) FROM timetable.classes
UNION ALL SELECT 'class_instructors', COUNT(*) FROM timetable.class_instructors
UNION ALL SELECT 'class_time_options', COUNT(*) FROM timetable.class_time_options
UNION ALL SELECT 'students', COUNT(*) FROM timetable.students;


Finally, this SELECT query reports row counts for each table. These are simple verification snapshots to confirm that every CSV was loaded as expected and to spot obvious issues (zero rows in a critical table or mismatches between related tables).

In [ ]:
SELECT 'rooms' AS t, COUNT(*) FROM timetable.rooms
UNION ALL SELECT 'room_sharing_patterns', COUNT(*) FROM timetable.room_sharing_patterns
UNION ALL SELECT 'room_sharing_departments', COUNT(*) FROM timetable.room_sharing_departments
UNION ALL SELECT 'classes', COUNT(*) FROM timetable.classes
UNION ALL SELECT 'class_instructors', COUNT(*) FROM timetable.class_instructors
UNION ALL SELECT 'class_room_options', COUNT(*) FROM timetable.class_room_options
UNION ALL SELECT 'class_time_options', COUNT(*) FROM timetable.class_time_options
UNION ALL SELECT 'instructors', COUNT(*) FROM timetable.instructors
UNION ALL SELECT 'constraints', COUNT(*) FROM timetable.constraints
UNION ALL SELECT 'constraint_classes', COUNT(*) FROM timetable.constraint_classes
UNION ALL SELECT 'students', COUNT(*) FROM timetable.students
UNION ALL SELECT 'student_offerings', COUNT(*) FROM timetable.student_offerings
UNION ALL SELECT 'student_classes', COUNT(*) FROM timetable.student_classes
UNION ALL SELECT 'student_prohibited_classes', COUNT(*) FROM timetable.student_prohibited_classes;


This part creates a scheduled_meetings table to store class meetings and enforces no double-booking. Each row ties a class_id to a room_id on a specific day_of_week, with a start_slot and length_slots. It generates a half-open integer range (slot_range) from start_slot to start_slot + length_slots to represent the occupied time. Constraints ensure valid days (0-6) and positive times, with foreign keys to classes (cascade on delete) and rooms (restrict delete). Finally, the GiST exclusion constraint no_room_time_overlap uses equality on room_id and day_of_week and the “overlaps” operator (&&) on slot_range so that any INSERT/UPDATE that would create overlapping time ranges in the same room on the same day is rejected.

In [ ]:
CREATE EXTENSION IF NOT EXISTS btree_gist;

CREATE TABLE scheduled_meetings (
  class_id VARCHAR(50) NOT NULL REFERENCES classes(class_id) ON DELETE CASCADE,
  room_id  VARCHAR(50) NOT NULL REFERENCES rooms(room_id) ON DELETE RESTRICT,
  day_of_week  SMALLINT NOT NULL CHECK (day_of_week BETWEEN 0 AND 6), -- 0=Sun to 6=Sat
  start_slot INTEGER NOT NULL CHECK (start_slot >= 0),
  length_slots INTEGER NOT NULL CHECK (length_slots > 0),
  slot_range int4range GENERATED ALWAYS AS (int4range(start_slot, start_slot + length_slots, '[)')) STORED,

  PRIMARY KEY (class_id, day_of_week, start_slot)
);

-- No two meetings should overlap in the same room on the same day
ALTER TABLE scheduled_meetings
ADD CONSTRAINT no_room_time_overlap
EXCLUDE USING gist (
  room_id WITH =,
  day_of_week WITH =,
  slot_range WITH &&
);

-- Priority: 1 = student, 2 = instructor, 3 = administrator
ALTER TABLE scheduled_meetings
  ADD COLUMN IF NOT EXISTS priority SMALLINT NOT NULL DEFAULT 1 CHECK (priority BETWEEN 1 AND 3);

-- Keep overlap rule consistent (no priority in the constraint)
ALTER TABLE scheduled_meetings DROP CONSTRAINT IF EXISTS no_room_time_overlap;

ALTER TABLE scheduled_meetings
ADD CONSTRAINT no_room_time_overlap
EXCLUDE USING gist (
  room_id     WITH =,
  day_of_week WITH =,
  slot_range  WITH &&
);


This block defines and attaches a trigger function that enforces logical room–class capacity rules. The PL/pgSQL function enforce_capacity_ok() runs automatically before any insert or update on scheduled_meetings. It looks up the chosen room’s capacity from rooms and the class’s enrollment limit from classes. If either record is missing, it raises an error. Then it checks that the room’s capacity is not smaller than the class limit (so every student can fit) and not excessively larger than 1.5x that limit (to avoid waste of space). If either condition fails, the trigger aborts the transaction with a clear exception message. The accompanying CREATE TRIGGER statement binds this check so that any attempt to assign or change a class–room pairing that violates size constraints is rejected automatically at the database level.

In [ ]:
-- Make sure we are talking about the right schema
CREATE SCHEMA IF NOT EXISTS timetable;
SET search_path = timetable;

-- Recreate the function with fully-qualified table names
CREATE OR REPLACE FUNCTION timetable.enforce_capacity_ok()
RETURNS trigger
LANGUAGE plpgsql AS $$
DECLARE
  room_cap   integer;
  class_lim  integer;
BEGIN
  -- Look up capacity from timetable.rooms
  SELECT r.capacity
    INTO room_cap
  FROM timetable.rooms r
  WHERE r.room_id = NEW.room_id;

  IF room_cap IS NULL THEN
    RAISE EXCEPTION 'Room % not found in timetable.rooms', NEW.room_id;
  END IF;

  -- Look up class_limit from timetable.classes
  SELECT c.class_limit
    INTO class_lim
  FROM timetable.classes c
  WHERE c.class_id = NEW.class_id;

  IF class_lim IS NULL THEN
    RAISE EXCEPTION 'Class % not found in timetable.classes', NEW.class_id;
  END IF;

  -- Lower bound: room must fit class
  IF room_cap < class_lim THEN
    RAISE EXCEPTION 'Capacity violation: room % cap % < class % limit %',
      NEW.room_id, room_cap, NEW.class_id, class_lim;
  END IF;

  -- Upper bound to avoid huge waste
  IF room_cap > class_lim * 1.5 THEN
    RAISE EXCEPTION 'Room % is too large for class % (cap %, limit %)',
      NEW.room_id, NEW.class_id, room_cap, class_lim;
  END IF;

  RETURN NEW;
END;
$$;

-- Reattach trigger to scheduled_meetings
DROP TRIGGER IF EXISTS trg_capacity_ok ON timetable.scheduled_meetings;

CREATE TRIGGER trg_capacity_ok
BEFORE INSERT OR UPDATE OF room_id, class_id
ON timetable.scheduled_meetings
FOR EACH ROW
EXECUTE FUNCTION timetable.enforce_capacity_ok();


This function, schedule_class, is a utility function that automates inserting class meeting times into the scheduled_meetings table based on a weekly schedule pattern. It accepts a class ID, room ID, a days_mask string ('0111100' for Monday-Friday), a start time slot, and the duration in slots. The loop iterates through days of the week (0-6) and, for each day marked as active ('1' in the mask), inserts a record specifying that the class meets in the given room at that time. Each insert automatically triggers the overlap and capacity checks you defined earlier, so if any day’s meeting would violate room capacity or double-book the room, the entire transaction rolls back, ensuring only valid, conflict-free schedules are committed.

In [ ]:

-- Usage:
-- Student booking (default priority=1)
-- SELECT schedule_class('STUDY-GRP-1', 'ENG-241', '0111100', 90, 6);
-- Instructor booking (preempts students if overlapping)
-- SELECT schedule_class('CS101-LEC', 'ENG-241', '0111100', 90, 6, 2);
-- Admin booking (preempts students & instructors if overlapping)
-- SELECT schedule_class('ADMIN-ALLOC', 'ENG-241', '0111100', 90, 6, 3);


SET search_path = timetable, public;

CREATE OR REPLACE FUNCTION timetable.schedule_class(
  p_class_id   text,
  p_room_id    text,
  p_days_mask  text,     -- '0111100' (Mon..Sun)
  p_start_slot integer,
  p_length     integer,
  p_priority   integer DEFAULT 1  -- 1=student, 2=instructor, 3=admin
)
RETURNS void
LANGUAGE plpgsql
AS $$
DECLARE
  d        smallint;
  v_range  int4range := int4range(p_start_slot, p_start_slot + p_length, '[)');
BEGIN
  FOR d IN 0..6 LOOP
    IF substr(p_days_mask, d + 1, 1) = '1' THEN
      -- Remove lower-priority overlaps in the same room/day/time
      DELETE FROM scheduled_meetings
       WHERE room_id = p_room_id
         AND day_of_week = d
         AND slot_range && v_range
         AND priority < p_priority;

      -- Insert one meeting for this day; capacity + overlap constraints will run
      INSERT INTO scheduled_meetings (
        class_id, room_id, day_of_week, start_slot, length_slots, priority
      )
      VALUES (
        p_class_id, p_room_id, d, p_start_slot, p_length, p_priority
      );
    END IF;
  END LOOP;
END;
$$;


This block implements a full complaints subsystem for rooms. First it defines a complaint_status enum to track lifecycle (open, in_progress, resolved, dismissed). The room_complaints table then stores each report with a surrogate key, the target room_id (cascading delete if a room is removed), and exactly one reporter, either a student or an instructor, enforced by CHECK (num_nonnulls(...) = 1). It captures anonymity, a short title, detailed description, a normalized priority (1-3), and audit timestamps (created_at, updated_at, optional resolved_at) plus optional resolved_notes. A lightweight trigger trg_touch_room_complaints updates updated_at on every row change. Finally, two indexes optimize common queries: by (room_id, status) for “open complaints for room X,” and (status, priority, created_at DESC) for dashboards that surface the newest, highest-priority issues first.

In [ ]:
CREATE TYPE complaint_status AS ENUM ('new','open','in_progress','resolved','dismissed');

CREATE TABLE room_complaints (
  complaint_id BIGSERIAL PRIMARY KEY,

  room_id      VARCHAR(50) NOT NULL
               REFERENCES rooms(room_id) ON DELETE CASCADE,

  -- Exactly one of these must be non-null:
  reporter_student_id    VARCHAR(50)
               REFERENCES students(student_id) ON DELETE SET NULL,
  reporter_instructor_id VARCHAR(50)
               REFERENCES instructors(instructor_id) ON DELETE SET NULL,

  is_anonymous  BOOLEAN NOT NULL DEFAULT FALSE,
  title         VARCHAR(120) NOT NULL,
  description   TEXT NOT NULL,

  -- 1 = high, 2 = normal, 3 = low
  priority      SMALLINT NOT NULL DEFAULT 2,

  status        complaint_status NOT NULL DEFAULT 'open',

  created_at    TIMESTAMPTZ NOT NULL DEFAULT now(),
  updated_at    TIMESTAMPTZ NOT NULL DEFAULT now(),
  resolved_at   TIMESTAMPTZ,
  resolved_notes TEXT,

  CHECK (num_nonnulls(reporter_student_id, reporter_instructor_id) = 1),
  CHECK (priority BETWEEN 1 AND 3)
);


CREATE OR REPLACE FUNCTION touch_room_complaints_updated_at()
RETURNS trigger
LANGUAGE plpgsql AS $$
BEGIN
  NEW.updated_at := now();
  RETURN NEW;
END;
$$;

DROP TRIGGER IF EXISTS trg_touch_room_complaints ON room_complaints;

CREATE TRIGGER trg_touch_room_complaints
BEFORE UPDATE ON room_complaints
FOR EACH ROW
EXECUTE FUNCTION touch_room_complaints_updated_at();

-- Give complaints a default title when none is provided
ALTER TABLE room_complaints
  ALTER COLUMN title SET DEFAULT 'General room complaint';

-- Fix any rows with title NULL
UPDATE room_complaints
SET title = 'General room complaint'
WHERE title IS NULL;

ALTER TABLE room_complaints
  DROP CONSTRAINT room_complaints_priority_check;

ALTER TABLE room_complaints
  ADD CONSTRAINT room_complaints_priority_check
  CHECK (priority BETWEEN 1 AND 5);

-- complaint_id -> room_complaint_id
ALTER TABLE room_complaints
  RENAME COLUMN complaint_id TO room_complaint_id;

-- resolved_notes -> admin_notes
ALTER TABLE room_complaints
  RENAME COLUMN resolved_notes TO admin_notes;

ALTER TABLE room_complaints
  ALTER COLUMN status SET DEFAULT 'new';


CREATE INDEX ON room_complaints (room_id, status);
CREATE INDEX ON room_complaints (status, priority, created_at DESC);


This section adds two stored procedures that manage complaints in the database. The first function, add_room_complaint, provides a consistent, secure way to insert new complaints into the room_complaints table. It accepts parameters for the room, title, description, priority, optional reporter (either student or instructor), and anonymity flag, then inserts the record and returns the new complaint’s ID. The second function, set_complaint_status, is used to update an existing complaint’s status, changing it to “in progress,” “resolved,” or “dismissed”—and automatically timestamps the resolution when applicable. Together, these functions form the operational API for creating and managing complaints, letting users file new reports and administrators update their progress or resolution directly through SQL commands.

In [ ]:
-- Create a complaint and return its room_complaint_id
CREATE OR REPLACE FUNCTION add_room_complaint(
  p_room_id        VARCHAR,
  p_title          VARCHAR,
  p_description    TEXT,
  p_priority       INTEGER DEFAULT 2,   -- 1=high, 2=normal, 3=low
  p_student_id     VARCHAR DEFAULT NULL,
  p_instructor_id  VARCHAR DEFAULT NULL,
  p_anonymous      BOOLEAN DEFAULT FALSE
) RETURNS BIGINT
LANGUAGE plpgsql AS $$
DECLARE
  v_id BIGINT;
BEGIN
  -- Ensure exactly one reporter is provided
  IF (p_student_id IS NULL) = (p_instructor_id IS NULL) THEN
    RAISE EXCEPTION
      'Exactly one of p_student_id or p_instructor_id must be non-null';
  END IF;

  -- Business rule for priority.
  -- Table allows 1–5, but we currently only accept 1–3 here.
  IF p_priority NOT BETWEEN 1 AND 3 THEN
    RAISE EXCEPTION
      'p_priority must be between 1 and 3 (1=high, 2=normal, 3=low). Got %',
      p_priority;
  END IF;

  INSERT INTO room_complaints (
    room_id,
    title,
    description,
    priority,
    reporter_student_id,
    reporter_instructor_id,
    is_anonymous
    -- status will default to 'new'
  )
  VALUES (
    p_room_id,
    COALESCE(p_title, 'General room complaint'),
    p_description,
    p_priority,          -- implicitly cast to SMALLINT for the column
    p_student_id,
    p_instructor_id,
    p_anonymous
  )
  RETURNING room_complaint_id INTO v_id;   -- <== match renamed PK

  RETURN v_id;
END;
$$;


-- Set status for a complaint, by room_complaint_id
CREATE OR REPLACE FUNCTION set_complaint_status(
  p_room_complaint_id BIGINT,
  p_status            complaint_status,
  p_admin_notes       TEXT DEFAULT NULL
) RETURNS VOID
LANGUAGE plpgsql AS $$
BEGIN
  UPDATE room_complaints
     SET status      = p_status,
         resolved_at = CASE
                         WHEN p_status IN ('resolved','dismissed')
                           THEN now()
                         ELSE NULL
                       END,
         admin_notes = p_admin_notes
   WHERE room_complaint_id = p_room_complaint_id;
   -- updated_at is handled by the trigger
END;
$$;



-- Seed data for demo
INSERT INTO rooms (room_id) VALUES
  ('ENG-241'),
  ('SCI-102')
ON CONFLICT (room_id) DO NOTHING;

INSERT INTO students (student_id) VALUES
  ('s123456')
ON CONFLICT (student_id) DO NOTHING;

INSERT INTO instructors (instructor_id) VALUES
  ('i7890')
ON CONFLICT (instructor_id) DO NOTHING;


-- Example usage
-- Student files a complaint and we capture its room_complaint_id
SELECT add_room_complaint(
  p_room_id        => 'ENG-241',
  p_title          => 'Projector not turning on',
  p_description    => 'Tried multiple HDMI cables; fan spins up, no image.',
  p_priority       => 1,           -- high
  p_student_id     => 's123456',
  p_instructor_id  => NULL,
  p_anonymous      => FALSE
) AS room_complaint_id;


-- Instructor files a complaint (anonymous)
SELECT add_room_complaint(
  p_room_id        => 'SCI-102',
  p_title          => 'Broken chair at back row',
  p_description    => 'Third from left, back row.',
  p_priority       => 2,          -- normal
  p_student_id     => NULL,
  p_instructor_id  => 'i7890',
  p_anonymous      => TRUE
) AS room_complaint_id;


-- Mark a specific complaint as resolved with notes
-- (replace 42 with the actual room_complaint_id returned above)
SELECT set_complaint_status(
  p_room_complaint_id => 100,
  p_status            => 'resolved',
  p_admin_notes       => 'Replaced projector lamp on 2025-11-22'
);


Script for extracting csvs from xml:

In [ ]:
#!/usr/bin/env python3
"""
Extract UniTime-style university timetabling XML into CSVs.

Writes to ./out_csv:
  - rooms.csv
  - room_sharing_patterns.csv
  - room_sharing_departments.csv
  - classes.csv
  - class_instructors.csv
  - class_room_options.csv
  - class_time_options.csv
  - instructors.csv
  - constraints.csv
  - constraint_classes.csv
  - students.csv
  - student_offerings.csv
  - student_classes.csv
  - student_prohibited_classes.csv
"""

import csv
import os
import xml.etree.ElementTree as ET
from typing import Optional, Tuple

# Small helpers

def ensure_dir(path: str):
    os.makedirs(path, exist_ok=True)

def strip_ns(tag: str) -> str:
    if "}" in tag:
        return tag.split("}", 1)[1]
    return tag

def iter_children(elem, wanted_tag: str):
    for child in list(elem):
        if strip_ns(child.tag) == wanted_tag:
            yield child

def to_bool(text: Optional[str]) -> Optional[bool]:
    if text is None:
        return None
    t = text.strip().lower()
    if t in ("true", "t", "1", "yes", "y"):
        return True
    if t in ("false", "f", "0", "no", "n"):
        return False
    return None

def to_int(text: Optional[str]) -> Optional[int]:
    try:
        return int(text) if text is not None else None
    except ValueError:
        return None

def to_float(text: Optional[str]) -> Optional[float]:
    try:
        return float(text) if text is not None else None
    except ValueError:
        return None

def split_location(loc: Optional[str]) -> Tuple[Optional[int], Optional[int]]:
    if not loc or "," not in loc:
        return None, None
    x, y = loc.split(",", 1)
    return to_int(x), to_int(y)

class CsvWriter:
    def __init__(self, path: str, header: list[str]):
        self.f = open(path, "w", newline="", encoding="utf-8")
        self.w = csv.writer(self.f)
        self.w.writerow(header)
    def writerow(self, row):
        self.w.writerow(row)
    def close(self):
        self.f.close()

# Main extraction

def extract(xml_path: str, out_dir: str):
    ensure_dir(out_dir)

    rooms_w = CsvWriter(os.path.join(out_dir, "rooms.csv"),
                        ["room_id", "capacity", "location_x", "location_y", "has_constraints"])
    room_patterns_w = CsvWriter(os.path.join(out_dir, "room_sharing_patterns.csv"),
                                ["room_id", "unit_slots", "free_for_all_char", "not_available_char", "pattern_text"])
    room_depts_w = CsvWriter(os.path.join(out_dir, "room_sharing_departments.csv"),
                             ["room_id", "digit_char", "department_id"])

    classes_w = CsvWriter(os.path.join(out_dir, "classes.csv"),
                          ["class_id", "offering_id", "config_id", "subpart_id",
                           "committed", "class_limit", "scheduler", "dates_mask"])

    instructors_w = CsvWriter(os.path.join(out_dir, "instructors.csv"),
                              ["instructor_id"])

    class_instr_w = CsvWriter(os.path.join(out_dir, "class_instructors.csv"),
                              ["class_id", "instructor_id"])
    class_roomopt_w = CsvWriter(os.path.join(out_dir, "class_room_options.csv"),
                                ["class_id", "room_id", "pref"])
    class_timeopt_w = CsvWriter(os.path.join(out_dir, "class_time_options.csv"),
                                ["class_id", "days_mask", "start_slot", "length_slots", "pref"])

    constraints_w = CsvWriter(os.path.join(out_dir, "constraints.csv"),
                              ["pk", "external_id", "type", "pref_raw", "pref_numeric"])
    constr_classes_w = CsvWriter(os.path.join(out_dir, "constraint_classes.csv"),
                                 ["constraint_pk", "order_index", "class_id"])

    students_w = CsvWriter(os.path.join(out_dir, "students.csv"),
                           ["student_id"])
    stud_offerings_w = CsvWriter(os.path.join(out_dir, "student_offerings.csv"),
                                 ["student_id", "offering_id", "weight"])
    stud_classes_w = CsvWriter(os.path.join(out_dir, "student_classes.csv"),
                               ["student_id", "class_id"])
    stud_prohibited_w = CsvWriter(os.path.join(out_dir, "student_prohibited_classes.csv"),
                                  ["student_id", "class_id"])

    next_constraint_pk = 1
    seen_instructor_ids: set[str] = set()

    # Stream parse
    context = ET.iterparse(xml_path, events=("end",))
    for event, elem in context:
        tag = strip_ns(elem.tag)

        # ROOMS
        if tag == "room":
            # Only process top-level room definitions (they have 'capacity')
            if "capacity" not in elem.attrib:
                # This is a <room> inside <class> (i.e., a room option).
                # Do NOT clear it here; the <class> handler will read it.
                continue

            rid = elem.attrib.get("id")
            capacity = to_int(elem.attrib.get("capacity"))
            has_constr = to_bool(elem.attrib.get("constraint"))
            locx, locy = split_location(elem.attrib.get("location"))
            rooms_w.writerow([rid, capacity, locx, locy, has_constr])

            sharing = next(iter_children(elem, "sharing"), None)
            if sharing is not None:
                pattern_el = next(iter_children(sharing, "pattern"), None)
                unit_slots = to_int(pattern_el.attrib.get("unit")) if pattern_el is not None else None
                pattern_text = (pattern_el.text or "").strip() if pattern_el is not None else None

                ffa_el = next(iter_children(sharing, "freeForAll"), None)
                not_av_el = next(iter_children(sharing, "notAvailable"), None)
                free_for_all_char = ffa_el.attrib.get("value") if ffa_el is not None else None
                not_available_char = not_av_el.attrib.get("value") if not_av_el is not None else None
                room_patterns_w.writerow([rid, unit_slots, free_for_all_char, not_available_char, pattern_text])

                for dept in iter_children(sharing, "department"):
                    digit_char = dept.attrib.get("value")
                    dept_id = dept.attrib.get("id")
                    room_depts_w.writerow([rid, digit_char, dept_id])

            # safe to clear only real room definitions
            elem.clear()


        # CLASS DEFINITIONS (only real ones under <classes>)
        elif tag == "class":
            # Heuristic: only treat as a real "class definition" if it has class-def attributes
            is_definition = (
                ("classLimit" in elem.attrib) or
                ("offering" in elem.attrib) or
                (next(iter_children(elem, "time"), None) is not None)
            )
            if not is_definition:
                # This is a reference (likely under <students> or <constraint>)
                # DO NOT clear: parent (<student> / <constraint>) still needs it.
                continue

            cid = elem.attrib.get("id")
            offering = elem.attrib.get("offering")
            config = elem.attrib.get("config")
            subpart = elem.attrib.get("subpart")
            committed = to_bool(elem.attrib.get("committed"))
            class_limit = to_int(elem.attrib.get("classLimit"))
            scheduler = to_int(elem.attrib.get("scheduler"))
            dates_mask = elem.attrib.get("dates")
            classes_w.writerow([cid, offering, config, subpart, committed, class_limit, scheduler, dates_mask])

            for ins in iter_children(elem, "instructor"):
                instr_id = ins.attrib.get("id")
                if instr_id:
                    class_instr_w.writerow([cid, instr_id])
                    if instr_id not in seen_instructor_ids:
                        seen_instructor_ids.add(instr_id)

            for r in iter_children(elem, "room"):
                room_id = r.attrib.get("id")
                pref = to_float(r.attrib.get("pref"))
                class_roomopt_w.writerow([cid, room_id, pref])

            for t in iter_children(elem, "time"):
                days_mask = t.attrib.get("days")
                start_slot = to_int(t.attrib.get("start"))
                length_slots = to_int(t.attrib.get("length"))
                pref = to_float(t.attrib.get("pref"))
                class_timeopt_w.writerow([cid, days_mask, start_slot, length_slots, pref])

            # safe to clear only these full definitions
            elem.clear()

        # CONSTRAINTS + class membership
        elif tag == "constraint":
            external_id = elem.attrib.get("id")
            ctype = elem.attrib.get("type")
            pref_raw = elem.attrib.get("pref")
            try:
                pref_numeric = float(pref_raw)
            except (TypeError, ValueError):
                pref_numeric = None

            pk = next_constraint_pk
            next_constraint_pk += 1
            constraints_w.writerow([pk, external_id, ctype, pref_raw, pref_numeric])

            order_idx = 1
            for c in iter_children(elem, "class"):
                cid = c.attrib.get("id")
                if cid:
                    constr_classes_w.writerow([pk, order_idx, cid])
                    order_idx += 1

            elem.clear()

        # STUDENTS + links
        elif tag == "student":
            sid = elem.attrib.get("id")
            if sid:
                students_w.writerow([sid])

                for off in iter_children(elem, "offering"):
                    oid = off.attrib.get("id")
                    weight = to_float(off.attrib.get("weight"))
                    stud_offerings_w.writerow([sid, oid, weight])

                for c in iter_children(elem, "class"):
                    cid = c.attrib.get("id")
                    if cid:
                        stud_classes_w.writerow([sid, cid])

                for pc in iter_children(elem, "prohibited-class"):
                    cid = pc.attrib.get("id")
                    if cid:
                        stud_prohibited_w.writerow([sid, cid])

            elem.clear()

    for iid in sorted(seen_instructor_ids):
        instructors_w.writerow([iid])

    # Close writers
    for w in (rooms_w, room_patterns_w, room_depts_w,
              classes_w, class_instr_w, class_roomopt_w, class_timeopt_w,
              constraints_w, constr_classes_w,
              students_w, stud_offerings_w, stud_classes_w, stud_prohibited_w):
        w.close()

def main():
    # Always work under the current directory
    cwd = os.getcwd()

    # Input XML (change the file name below if yours is different, but I assume you downloaded it from where we did)
    xml_path = os.path.join(cwd, "pu-fal07-c8.xml")
    if not os.path.exists(xml_path):
        raise FileNotFoundError(f"XML not found at: {xml_path}")

    # Output directory (fixed name; no CLI params)
    out_dir = os.path.join(cwd, "out_csv")
    os.makedirs(out_dir, exist_ok=True)

    extract(xml_path, out_dir)
    print(f"CSVs written to: {out_dir}")

if __name__ == "__main__":
    main()

FileNotFoundError: XML not found at: /content/pu-fal07-c8.xml

Below is the setup for the MySQL dump backup. **Still need to fully test with all the test data**